In [1]:
!conda install pandas

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.1.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Retrieving notices: ...working... done


In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/processed/alura_real_estate.csv')
df.head()

,square_meters,year_built,garage_cars,price
0,159.0,2003,2,208500
1,117.0,1976,2,181500
2,166.0,2001,2,223500
3,160.0,1915,3,140000
4,204.0,2000,3,250000


In [3]:
x = df.drop('price', axis=1)
y = df['price']

In [6]:
!conda install scikit-learn -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.1.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Retrieving notices: ...working... done


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [5]:
X_train.head()

,square_meters,year_built,garage_cars
135,156.0,1970,2
1452,100.0,2005,2
762,144.0,2009,2
932,177.0,2006,3
435,154.0,1996,2


In [6]:
y_train

135     174000
1452    145000
762     215200
932     320000
435     212000
         ...  
1095    176432
1130    135000
1294    115000
860     189950
1126    174000
Name: price, Length: 1022, dtype: int64

# Linear Regression

In [7]:
import mlflow

mlflow.set_experiment('house-prices')

<Experiment: artifact_location='file:///home/julio/jobs/alura/mlflow-en/mlflow/notebooks/mlruns/747829818802971770', creation_time=1676161451122, experiment_id='747829818802971770', last_update_time=1676161451122, lifecycle_stage='active', name='house-prices', tags={}>

In [8]:
from sklearn.linear_model import LinearRegression

In [9]:
mlflow.start_run()

<ActiveRun: >

In [10]:

lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [27]:
!conda install -c conda-forge mlflow['xgboost','scikit-learn'] -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.1.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Retrieving notices: ...working... done


In [11]:
mlflow.sklearn.log_model(lr,'lr')

/home/julio/anaconda3/envs/py10/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [12]:
lr.predict(X_test)[0]

119279.77015440399

In [13]:
from sklearn.metrics import mean_squared_error, r2_score

In [14]:
lr_predicted = lr.predict(X_test)

In [15]:
import math

mse = mean_squared_error(y_test, lr_predicted)
r2 = r2_score(y_test, lr_predicted)
rmse = math.sqrt(mse)
mlflow.log_metric("mse",mse)
mlflow.log_metric("rmse",rmse)
mlflow.log_metric("r2",r2)

In [16]:
mlflow.end_run()

In [17]:
print(f' rmse: {rmse}, mse: {mse}, r2 {r2}')

 rmse: 45592.399782518485, mse: 2078666917.9289918, r2 0.7021153642898047


# XGBoost

In [18]:
!conda install xgboost -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 22.9.0
  latest version: 23.1.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/julio/anaconda3/envs/py10

  added / updated specs:
    - xgboost


The following NEW packages will be INSTALLED:

  _py-xgboost-mutex  pkgs/main/linux-64::_py-xgboost-mutex-2.0-cpu_0 None
  libxgboost         pkgs/main/linux-64::libxgboost-1.7.3-h6a678d5_0 None
  py-xgboost         pkgs/main/linux-64::py-xgboost-1.7.3-py310h06a4308_0 None
  xgboost            pkgs/main/linux-64::xgboost-1.7.3-py310h06a4308_0 None


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Retrieving notices: ...working... done


In [18]:
from xgboost import XGBRegressor

In [19]:
with mlflow.start_run():
    xgb = XGBRegressor(random_state=42)
    xgb.fit(X_train, y_train)
    mlflow.xgboost.log_model(xgb, "xgb")
    xgb_predicted = xgb.predict(X_test)
    mse = mean_squared_error(y_test, xgb_predicted)
    r2 = r2_score(y_test, xgb_predicted)
    rmse = math.sqrt(mse)
    print(f' rmse: {rmse}, mse: {mse}, r2 {r2}')
    mlflow.log_metric("mse",mse)
    mlflow.log_metric("rmse",rmse)
    mlflow.log_metric("r2",r2)

 rmse: 39380.40189141977, mse: 1550816053.129738, r2 0.7777593557411607


## Testing params

In [20]:
with mlflow.start_run():
    xgb = XGBRegressor(learning_rate=0.2, n_estimators=50, random_stage=42)
    xgb.fit(X_train, y_train)
    mlflow.xgboost.log_model(xgb, "xgb")
    xgb_predicted = xgb.predict(X_test)
    mse = mean_squared_error(y_test, xgb_predicted)
    r2 = r2_score(y_test, xgb_predicted)
    rmse = math.sqrt(mse)
    print(f' rmse: {rmse}, mse: {mse}, r2 {r2}')
    mlflow.log_metric("mse",mse)
    mlflow.log_metric("rmse",rmse)
    mlflow.log_metric("r2",r2)

[21:36:10] WARNING: /croot/xgboost-split_1675457761144/work/src/learner.cc:767: 
Parameters: { "random_stage" } are not used.

 rmse: 37238.789724353286, mse: 1386727460.1346002, r2 0.8012741720529797


In [25]:
xgb_predicted = xgb.predict(X_test)
mse = mean_squared_error(y_test, xgb_predicted)
r2 = r2_score(y_test, xgb_predicted)
rmse = math.sqrt(mse)
print(f' rmse: {rmse}, mse: {mse}, r2 {r2}')

 rmse: 37238.789724353286, mse: 1386727460.1346002, r2 0.8012741720529797


In [21]:
experiment = mlflow.get_experiment_by_name('house-prices')
experiment

<Experiment: artifact_location='file:///home/julio/jobs/alura/mlflow-en/mlflow/notebooks/mlruns/747829818802971770', creation_time=1676161451122, experiment_id='747829818802971770', last_update_time=1676161451122, lifecycle_stage='active', name='house-prices', tags={}>

In [27]:
runs = mlflow.search_runs(experiment_names=['house-prices'])
runs

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.rmse,metrics.mse,metrics.r2,tags.mlflow.source.name,tags.mlflow.runName,tags.mlflow.log-model.history,tags.mlflow.user,tags.mlflow.source.type
0,404c0b5d2afc4aa59f342419781e0c3c,747829818802971770,FINISHED,file:///home/julio/jobs/alura/mlflow-en/mlflow...,2023-02-12 00:36:10.702000+00:00,2023-02-12 00:36:13.059000+00:00,37238.789724,1.386727e+09,0.801274,/home/julio/anaconda3/envs/py10/lib/python3.10...,judicious-tern-343,"[{""run_id"": ""404c0b5d2afc4aa59f342419781e0c3c""...",julio,LOCAL
1,47ab4072d7a74c4a9ca97740456b398f,747829818802971770,FINISHED,file:///home/julio/jobs/alura/mlflow-en/mlflow...,2023-02-12 00:35:37.499000+00:00,2023-02-12 00:35:39.663000+00:00,39380.401891,1.550816e+09,0.777759,/home/julio/anaconda3/envs/py10/lib/python3.10...,upbeat-panda-851,"[{""run_id"": ""47ab4072d7a74c4a9ca97740456b398f""...",julio,LOCAL
2,2e07863e694c480a9dca1e77a30d81a7,747829818802971770,FAILED,file:///home/julio/jobs/alura/mlflow-en/mlflow...,2023-02-12 00:35:30.979000+00:00,2023-02-12 00:35:30.992000+00:00,NaN,NaN,NaN,/home/julio/anaconda3/envs/py10/lib/python3.10...,crawling-dog-665,None,julio,LOCAL
3,098e1a6c257c4426894405be02e76440,747829818802971770,FINISHED,file:///home/julio/jobs/alura/mlflow-en/mlflow...,2023-02-12 00:29:24.506000+00:00,2023-02-12 00:32:17.181000+00:00,45592.399783,2.078667e+09,0.702115,/home/julio/anaconda3/envs/py10/lib/python3.10...,nebulous-boar-164,"[{""run_id"": ""098e1a6c257c4426894405be02e76440""...",julio,LOCAL
4,2007c778301f4c12819170e498ff1d8e,747829818802971770,RUNNING,file:///home/julio/jobs/alura/mlflow-en/mlflow...,2023-02-12 00:25:23.617000+00:00,NaT,NaN,NaN,NaN,/home/julio/anaconda3/envs/py10/lib/python3.10...,painted-loon-470,None,julio,LOCAL


In [28]:
mlflow.get_run('404c0b5d2afc4aa59f342419781e0c3c')

<Run: data=<RunData: metrics={'mse': 1386727460.1346002,
 'r2': 0.8012741720529797,
 'rmse': 37238.789724353286}, params={}, tags={'mlflow.log-model.history': '[{"run_id": "404c0b5d2afc4aa59f342419781e0c3c", '
                             '"artifact_path": "xgb", "utc_time_created": '
                             '"2023-02-12 00:36:10.772253", "flavors": '
                             '{"python_function": {"loader_module": '
                             '"mlflow.xgboost", "python_version": "3.10.9", '
                             '"data": "model.None", "env": {"conda": '
                             '"conda.yaml", "virtualenv": "python_env.yaml"}}, '
                             '"xgboost": {"xgb_version": "1.7.3", "data": '
                             '"model.None", "model_class": '
                             '"xgboost.sklearn.XGBRegressor", "model_format": '
                             'null, "code": null}}, "model_uuid": '
                             '"c0875ad01edf4922a13c38bd4